In [1]:
import json
from pprint import pprint
import tinydb as tdb
import requests

In [2]:
db = tdb.TinyDB('data/tinyDB.json')

In [3]:
wikidata = db.table('wikidata')

In [4]:
wikidataall = wikidata.all()

In [13]:
!mkdir wikidataImages

In [7]:
wikidataall[2]

{'article': 'https://fr.wikipedia.org/wiki/Ossuaire_de_Douaumont',
 'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Douaumont%20au%20Printemps.jpeg',
 'item': 'http://www.wikidata.org/entity/Q705054',
 'itemDescription': 'ossuaire à Fleury-devant-Douaumont (Meuse)',
 'merimee': 'PA55000002'}

In [16]:
def getWikiCode(itemUrl):
    items = itemUrl.split('/')
    return items[-1]

In [17]:
wikidataImages = [ (getWikiCode(wd['item']), wd['image']) for wd in wikidataall if 'image' in wd ]
print( len(wikidataImages) )

34503


In [20]:
path = 'wikidataImages/'
for name, url in wikidataImages[:50]:
    extension = url.split('.')[-1]
    name = name + '.' + extension
    
    r = requests.get(url)
    
    with open(path+name, 'wb') as f:
        f.write(r.content)
    
    print(name, end=', ')

Q611944.JPG, Q691375.JPG, Q705054.jpeg, Q386009.jpg, Q386604.JPG, Q188977.jpg, Q333942.jpg, Q378453.jpg, Q333931.JPG, Q692407.jpg, Q609842.JPG, Q724929.jpg, Q692768.jpg, Q377882.jpg, Q334060.jpg, Q608624.jpg, Q691089.jpg, Q688665.jpg, Q608423.jpg, Q722496.jpg, Q334011.JPG, Q334082.jpg, 

KeyboardInterrupt: 

In [15]:
wikidataImages[2]

('[Q705054](http://www.wikidata.org/entity/Q705054)',
 'http://commons.wikimedia.org/wiki/Special:FilePath/Douaumont%20au%20Printemps.jpeg')